<a href="https://colab.research.google.com/github/daisysong76/AI--Machine--learning/blob/main/Advanced_Multi_Agent_Educational_Content_Generation_System.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
from torch import nn
from transformers import (
    GPT2LMHeadModel, GPT2Tokenizer,
    T5ForConditionalGeneration, T5Tokenizer,
    BertModel, BertTokenizer,
    AdamW
)
import numpy as np
from dataclasses import dataclass
from typing import List, Dict, Tuple, Optional, Union
from collections import deque
import ray
from ray import tune
from ray.rllib.algorithms.ppo import PPO
import wandb
from sklearn.cluster import KMeans
from sentence_transformers import SentenceTransformer
import pytorch_lightning as pl
from torch.utils.data import Dataset, DataLoader
import tensorflow as tf
import gym
from gym import spaces
import pandas as pd
from tqdm import tqdm

@dataclass
class ContentExample:
    text: str
    topic: str
    difficulty: float
    target_age: int
    metadata: Dict
    rewards: Dict[str, float]

class RewardModel(nn.Module):
    """Advanced reward model using multi-aspect evaluation"""
    def __init__(self, model_name: str = 'bert-base-uncased'):
        super().__init__()
        self.bert = BertModel.from_pretrained(model_name)

        # Multiple reward heads for different aspects
        self.heads = nn.ModuleDict({
            'quality': nn.Linear(768, 1),
            'engagement': nn.Linear(768, 1),
            'educational_value': nn.Linear(768, 1),
            'age_appropriateness': nn.Linear(768, 1),
            'difficulty_match': nn.Linear(768, 1)
        })

        self.tokenizer = BertTokenizer.from_pretrained(model_name)

    def forward(self, texts: List[str]) -> Dict[str, torch.Tensor]:
        encodings = self.tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=512,
            return_tensors='pt'
        )

        outputs = self.bert(**encodings)
        pooled = outputs.pooler_output

        return {
            aspect: head(pooled)
            for aspect, head in self.heads.items()
        }

class ActiveLearningManager:
    """Manages active learning for content generation"""
    def __init__(self, embedding_model: str = 'all-MiniLM-L6-v2'):
        self.embedding_model = SentenceTransformer(embedding_model)
        self.uncertainty_threshold = 0.7
        self.diversity_weight = 0.3

    def select_samples_for_labeling(
        self,
        candidates: List[ContentExample],
        n_samples: int
    ) -> List[ContentExample]:
        # Get embeddings
        embeddings = self.embedding_model.encode(
            [c.text for c in candidates]
        )

        # Calculate uncertainty scores
        uncertainty_scores = self._calculate_uncertainty(candidates)

        # Calculate diversity scores using KMeans
        kmeans = KMeans(n_clusters=min(n_samples, len(candidates)))
        cluster_labels = kmeans.fit_predict(embeddings)

        # Combine uncertainty and diversity
        final_scores = (
            self.uncertainty_threshold * uncertainty_scores +
            self.diversity_weight * self._get_diversity_scores(
                embeddings,
                cluster_labels
            )
        )

        # Select top samples
        selected_indices = np.argsort(final_scores)[-n_samples:]
        return [candidates[i] for i in selected_indices]

    def _calculate_uncertainty(
        self,
        candidates: List[ContentExample]
    ) -> np.ndarray:
        # Implementation for uncertainty estimation
        return np.random.random(len(candidates))

    def _get_diversity_scores(
        self,
        embeddings: np.ndarray,
        cluster_labels: np.ndarray
    ) -> np.ndarray:
        # Calculate distance from cluster centers
        return np.random.random(len(embeddings))

class ContentGenerationEnv(gym.Env):
    """RL environment for content generation"""
    def __init__(
        self,
        base_model: GPT2LMHeadModel,
        tokenizer: GPT2Tokenizer,
        reward_model: RewardModel
    ):
        super().__init__()
        self.base_model = base_model
        self.tokenizer = tokenizer
        self.reward_model = reward_model

        # Define action and observation spaces
        vocab_size = self.tokenizer.vocab_size
        self.action_space = spaces.Discrete(vocab_size)
        self.observation_space = spaces.Box(
            low=0,
            high=vocab_size,
            shape=(512,),
            dtype=np.int64
        )

        self.max_steps = 100
        self.current_step = 0
        self.generated_tokens = []

    def reset(self):
        self.current_step = 0
        self.generated_tokens = []
        initial_token = self.tokenizer.bos_token_id
        self.generated_tokens.append(initial_token)
        return self._get_observation()

    def step(self, action):
        self.current_step += 1
        self.generated_tokens.append(action)

        done = self.current_step >= self.max_steps

        if done:
            reward = self._calculate_reward()
        else:
            reward = 0

        return self._get_observation(), reward, done, {}

    def _get_observation(self):
        obs = np.zeros(512, dtype=np.int64)
        obs[:len(self.generated_tokens)] = self.generated_tokens
        return obs

    def _calculate_reward(self):
        text = self.tokenizer.decode(self.generated_tokens)
        rewards = self.reward_model([text])
        return sum(r.mean().item() for r in rewards.values())

class MultiAgentContentGenerator:
    """Multi-agent system for content generation"""
    def __init__(
        self,
        num_agents: int = 3,
        base_model: str = 'gpt2-medium'
    ):
        ray.init()

        # Initialize models
        self.base_model = GPT2LMHeadModel.from_pretrained(base_model)
        self.tokenizer = GPT2Tokenizer.from_pretrained(base_model)
        self.reward_model = RewardModel()

        # Initialize RL agents
        self.agents = [
            PPO(
                env=ContentGenerationEnv,
                config={
                    "env_config": {
                        "base_model": self.base_model,
                        "tokenizer": self.tokenizer,
                        "reward_model": self.reward_model
                    }
                }
            )
            for _ in range(num_agents)
        ]

        self.active_learning = ActiveLearningManager()
        self.experience_buffer = deque(maxlen=10000)

    def train(
        self,
        num_iterations: int,
        samples_per_iteration: int = 100
    ):
        for iteration in range(num_iterations):
            # Generate content with each agent
            contents = []
            for agent in self.agents:
                content = self._generate_content(agent)
                contents.extend(content)

            # Select diverse samples for evaluation
            selected_samples = self.active_learning.select_samples_for_labeling(
                contents,
                samples_per_iteration
            )

            # Get rewards and update agents
            for sample in selected_samples:
                rewards = self.reward_model([sample.text])
                self.experience_buffer.append((sample, rewards))

                # Update each agent
                for agent in self.agents:
                    agent.train()

            # Periodically update reward model
            if iteration % 10 == 0:
                self._update_reward_model()

    def _generate_content(
        self,
        agent: PPO,
        num_samples: int = 10
    ) -> List[ContentExample]:
        contents = []
        for _ in range(num_samples):
            env = ContentGenerationEnv(
                self.base_model,
                self.tokenizer,
                self.reward_model
            )

            obs = env.reset()
            done = False
            while not done:
                action = agent.compute_single_action(obs)
                obs, reward, done, _ = env.step(action)

            text = self.tokenizer.decode(env.generated_tokens)
            contents.append(
                ContentExample(
                    text=text,
                    topic="",  # Add topic inference
                    difficulty=0.5,  # Add difficulty inference
                    target_age=12,  # Add age inference
                    metadata={},
                    rewards={}
                )
            )

        return contents

    def _update_reward_model(self):
        # Update reward model using experience buffer
        optimizer = AdamW(self.reward_model.parameters(), lr=1e-5)

        # Simple training loop
        for _ in range(100):
            samples = random.sample(
                self.experience_buffer,
                min(32, len(self.experience_buffer))
            )
            texts = [s[0].text for s in samples]
            true_rewards = torch.stack([
                torch.tensor([r.item() for r in s[1].values()])
                for s in samples
            ])

            predicted_rewards = self.reward_model(texts)
            loss = sum(
                nn.MSELoss()(pred.squeeze(), true_rewards[:, i])
                for i, pred in enumerate(predicted_rewards.values())
            )

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

    def generate_curriculum(
        self,
        topics: List[str],
        num_lessons: int,
        target_age: int
    ) -> List[ContentExample]:
        curriculum = []

        # Generate content using best performing agent
        best_agent = max(
            self.agents,
            key=lambda a: a.get_policy().model.get_metrics()["policy_reward_mean"]
        )

        for topic in topics:
            topic_lessons = []
            for _ in range(num_lessons):
                content = self._generate_content(
                    best_agent,
                    num_samples=5
                )
                # Select best content based on rewards
                best_content = max(
                    content,
                    key=lambda x: sum(x.rewards.values())
                )
                topic_lessons.append(best_content)

            # Order lessons by difficulty
            topic_lessons.sort(key=lambda x: x.difficulty)
            curriculum.extend(topic_lessons)

        return curriculum

if __name__ == "__main__":
    # Initialize system
    generator = MultiAgentContentGenerator(num_agents=3)

    # Example topics for curriculum generation
    topics = [
        "Introduction to Programming",
        "Variables and Data Types",
        "Control Structures",
        "Functions and Methods"
    ]

    # Train the system
    generator.train(num_iterations=100)

    # Generate curriculum
    curriculum = generator.generate_curriculum(
        topics=topics,
        num_lessons=3,
        target_age=14
    )

    # Save curriculum
    with open('generated_curriculum.json', 'w') as f:
        json.dump(
            [
                {
                    'topic': lesson.topic,
                    'content': lesson.text,
                    'difficulty': lesson.difficulty,
                    'target_age': lesson.target_age,
                    'metadata': lesson.metadata
                }
                for lesson in curriculum
            ],
            f,
            indent=2
        )

his advanced implementation includes several cutting-edge techniques:

Multi-Agent Learning:


Multiple RL agents generating content in parallel
Different agents can specialize in different aspects of content generation
Agents learn from each other's experiences
Competition and cooperation between agents


Sophisticated Reward Modeling:


Multi-aspect reward calculation
BERT-based content evaluation
Separate reward heads for different quality aspects
Continuous reward model updating


Active Learning:


Smart sample selection for evaluation
Diversity-based sampling
Uncertainty estimation
Efficient use of human feedback


Advanced Environment:


Custom gym environment for content generation
Structured action and observation spaces
Progressive reward calculation
Step-by-step content generation


Curriculum Generation:


Topic-based curriculum structure
Difficulty progression
Age-appropriate content filtering
Coherent lesson sequencing

To improve this system even further, you could:

Add Meta-Learning:

pythonCopy# Example meta-learning addition
class MetaLearningOptimizer(nn.Module):
    def __init__(self):
        super().__init__()
        self.meta_lstm = nn.LSTM(768, 768)
        
    def adapt_agents(self, agents, meta_batch):
        # Implement meta-learning updates
        pass

Implement Hierarchical RL:

pythonCopyclass HierarchicalAgent:
    def __init__(self):
        self.high_level_policy = PPO(...)  # For topic selection
        self.low_level_policy = PPO(...)   # For content generation

Add Self-Critical Training:

pythonCopydef self_critical_training(self, generated_content):
    baseline = self.generate_baseline()
    advantages = self.reward_model(generated_content) - self.reward_model(baseline)
    return advantages